In [7]:
!pip install --upgrade langchain langchain-community langgraph
!pip install langchain-ollama

## Simple Chatbot using Lang-Graph

In [8]:
from typing import List, Dict # to store the information for context
from langgraph.graph import StateGraph, START, END #stategraph is state machine
from langchain_ollama.llms import OllamaLLM
from langchain_community.llms import Ollama

In [9]:
#step 1: Define State
class State(Dict):
  messages: List[Dict[str,str]]

#step 2: Initialize StateGraph
graph_builder = StateGraph(State)

#Initialize the LLM
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")


#Define Chatbot Function
def chatbot(state: State):
  response = llm.invoke(state["messages"])
  state["messages"].append({"role": "assistant", "content": response})
  return{"messages": state["messages"]}

#Add nodes and edges
graph_builder.add_node("chatbot",chatbot)
graph_builder.add_edge(START,"chatbot")#connection building in chatbot
graph_builder.add_edge("chatbot",END) #connecting this to end node

graph = graph_builder.compile()#compile the graph

# Stream Updates
def stream_graph_updates(user_input:str):
  #Initialize the state with the user's input
  state ={"messages": [{"role": "user", "content": user_input}]}
  for event in graph.stream(state):
    for value in event.values():
      # Print the assistant's response
      print("Assistant:", value["messages"][-1]["content"])

# Run chatbot in loop
if __name__ == "__main__":
  while True:
    try:
      user_input = input("User: ")
      if user_input.lower() in ["quit", "exit" , "q"]:
        print("Goodbye!")
        break

      stream_graph_updates(user_input)
    except Exception as e:
      print(f"An error occurred: {e}")
      break

User:  HI, my name is Roger


Assistant: Nice to meet you, Roger! What brings you here today?


User:  i need to know what is the diffrence b/w langchain and lang graph?


Assistant: A great question!

In the field of natural language processing (NLP), both "langchain" and "langgraph" are terms related to linguistic structures, but they have distinct meanings:

**Langchain**: A language chain or langchain refers to a sequence of words or phrases that form a coherent sentence or phrase. In other words, it's a chain of linguistically connected units (words, phrases) that convey meaning. Langchains can be thought of as the building blocks of sentences, where each element is linked to the next through grammatical and semantic relationships.

For example: "The dog chased the cat" is a langchain because it forms a coherent sentence with clear relationships between the words.

**Langgraph**: A language graph or langgraph is a mathematical representation of linguistic structures, typically using graph theory. In this context, a graph is a set of nodes (vertices) connected by edges (arcs). Each node in the graph represents a word, phrase, or concept, and the edge

User:  thanks for the info


Assistant: You're welcome! However, I think there might be some confusion. This conversation just started, and I haven't provided you with any information yet. If you have any questions or topics you'd like to discuss, I'm here to help!


User:  quit


Goodbye!
